## 🔢 Diferencias_entre_duplicados_en_num_genes.ipynb

**Autor:** Johanna Atenea Carreon Baltazar  
**Contacto:** johannaatenea13@gmail.com
**Fecha de última modificación:** junio de 2025

---

### 🎯 Objetivo
Calcular las ** (distancias entre genes de familias expandidas) ** en cada genoma del grupo de estudio, medidas en **número de genes** que los separan. Este análisis busca detectar posibles patrones de agrupamiento o dispersión de duplicaciones a nivel funcional y estructural.

---

### 📥 Entradas requeridas

- `genes_secundarios_k_means.pkl` y `genes_secundarios_umbrales.pkl`: diccionarios con los identificadores de genes de familias expandidas por genoma.
- Archivos de anotación `.gbk` generados por RAST: utilizados para ordenar los genes por su localización y contar el número total de genes por genoma.

---

### 📤 Salidas generadas

- Diccionarios:
  - `resultados_diferencias_k_means`
  - `resultados_diferencias_umbrales`
  
  Cada uno contiene listas con las distancias entre genes duplicados consecutivos y la distancia circular dentro de cada genoma (en número de genes).

- Estadísticas descriptivas sobre la dispersión de duplicados en cada genoma, útiles para identificar duplicaciones localizadas o distribuidas de forma más uniforme.

---

### 🧰 Librerías requeridas

```python
import pickle
import csv
import pandas as pd
import numpy as np


# Pipeline
---


### 1. Importación de los archivos `genes_secundarios_k_means.pkl`y `genes_secundarios_umbrales.pkl`

Estos archivos contienen diccionarios con información sobre los genes secundarios. Las claves corresponden a los identificadores de los genomas, mientras que los valores son listas de identificadores de genes duplicados en cada genoma.

Un aspecto importante es que estos identificadores mantienen el orden en el que los genes están localizados dentro del genoma. Esta organización se puede verificar en detalle en los archivos .gbk, obtenidos tras la anotación de cada genoma con RAST.

**Ejemplo de estructura del diccionario:**

```python
genes_secundarios_umbrales = {
    2914041.10: [4092, 4899, 903, 1259, 3694, 2151, 936, 4764, 5875, 2666, 1072, 1013, 655, 1058, 5341, 977],
    ...
```
---
### 2. Ordenamos los identificadores de los genes duplicados para cada genoma.
---
### 3. Contamos los genes para cada genoma
De manera análoga al cálculo de diferencias entre genes duplicados en pares de bases, donde utilizamos la longitud del genoma en pares de bases para calcular la diferencia entre el primer y el último gen duplicado, en este caso, para determinar la diferencia entre estos genes en términos del número de genes que los separan, es necesario conocer el número total de genes en cada genoma.

Para obtener esta información, utilizamos archivos de anotación generados por RAST. A partir de estos archivos, aplicamos un filtrado que nos permitió contar el número total de genes en cada genoma, incluyendo tanto los genes codificantes de proteínas como aquellos que codifican ARN.

---
### 4. Calcular la diferencia entre genes duplicados de cada genoma en número de genes.
 Se calculan las diferencias entre los genes duplicados de cada genoma, esto se repite para cada método de clasificación(k_means y umbrales). Estas diferencias se almacenan en diccionarios  llamados `resultados_diferencias_k_means` y `resultados_diferencias_umbrales` respectivos , cuya estructura es la siguiente:
```
resultados_diferencias_k_means = {
    id_genoma: [diferencias entre genes duplicados],
    ...}
```
Tales diferencias se calculan con la función calcular_diferencias_genes, la cual en este caso mide las distancias en términos del número de genes que separan a cada par de genes duplicados dentro del mismo genoma. Esto se hace en dos pasos principales:
- Diferencias entre genes duplicados consecutivos  

Para cada par de genes duplicados consecutivos en la lista ordenada por su posición inicial  $p_{\text{start}}$, la diferencia se calcula como:

$$
d_i = \text{índice}(g_{i+1}) - \text{índice}(g_i)
$$

Donde $g_i$ y \$g_{i+1}$ son los genes duplicados consecutivos dentro del genoma y $d_i$ representa la cantidad de genes que los separan.

- Diferencia circular entre el último y el primer gen duplicado  

Como los genomas son circulares, también se considera la distancia entre el último y el primer gen duplicado:

$$
d_{\text{circular}} = (\text{índice}(g_1) + N - \text{índice}(g_n))
$$
Donde $g_1$ es el primer gen duplicado en la lista ordenada, $g_n$ es el último gen duplicado en la lista ordenada y  $N$ es el total de genes en el genoma.

---
### 4. Estadisticas descriptivas para cada uno de los diccionarios anteriores.

## Importar los diccionarios

In [2]:
import pickle
import pandas as pd
import numpy as np

### K-means

In [6]:
# Cargar el archivo .pkl
with open('genes_secundarios_k_means.pkl', 'rb') as file:
    genes_secundarios_k_means = pickle.load(file)

# Verificar el contenido
#print(type(genes_copia))  # Para saber qué tipo de objeto es
print(genes_secundarios_k_means)  # Muestra una parte del contenido

{2490939.1: [317, 321, 322, 334, 503, 608, 993, 1308, 1494, 1565, 2005, 2008, 2193, 2199, 2660, 2792, 2955, 3024, 3350, 3382, 3596, 3719, 3792, 3793, 3970, 4309, 4507, 4588, 4614, 4940, 5162, 5175, 5508, 5720, 5722, 5959, 6197, 6231], 103690.82: [665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2105, 2315, 2460, 2688, 2936, 2956, 3081, 3286, 3468, 3469, 4855, 4867, 4870, 5161, 5392, 5515, 5803, 5805, 5835], 211165.2: [198, 276, 365, 438, 653, 667, 1120, 1234, 1335, 1337, 1568, 1762, 1803, 2208, 2238, 2239, 2290, 2354, 2380, 2812, 2935, 3193, 3227, 3340, 3471, 3473, 3687, 3793, 3806, 3824, 3954, 3972, 4147, 4203, 4317, 4367, 4396, 4489, 4557, 4701, 4802, 4805, 4862, 5039, 5204, 5506, 6213, 6355, 6359, 6372, 6425, 6443, 6444, 6483, 6608, 6648, 6854, 6868, 6969, 7210], 1472755.9: [88, 430, 868, 915, 921, 1182, 1346, 1623, 1739, 2005, 2257, 3031, 3250, 3334, 3561, 4003, 4343, 4345, 4763, 5327, 5612, 5747, 6272, 6674, 6826, 7090, 7114, 7122, 7160, 7307], 2576904.6: [14, 833, 899, 1

### umbrales

In [5]:
# Cargar el archivo .pkl
with open('genes_secundarios_umbrales.pkl', 'rb') as file:
    genes_secundarios_umbrales = pickle.load(file)

# Verificar el contenido
#print(type(genes_copia))  # Para saber qué tipo de objeto es
print(genes_secundarios_umbrales)  # Muestra una parte del contenido

{2490939.1: [248, 317, 321, 322, 334, 378, 503, 525, 608, 895, 993, 1106, 1234, 1494, 1565, 1597, 2005, 2144, 2186, 2194, 2199, 2660, 2764, 2882, 2955, 3024, 3350, 3382, 3461, 3655, 3700, 3719, 3792, 3793, 3796, 3970, 4309, 4507, 4588, 4614, 4940, 5063, 5150, 5162, 5173, 5175, 5381, 5518, 5578, 5720, 5722, 5959, 6151, 6197, 6231, 6283, 6284, 6349], 103690.82: [98, 189, 190, 378, 469, 665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2086, 2105, 2315, 2460, 2688, 2766, 2936, 2956, 3002, 3081, 3195, 3286, 3468, 3469, 3472, 3552, 3998, 4086, 4564, 4646, 4855, 4867, 4870, 5161, 5220, 5392, 5435, 5515, 5579, 5611, 5619, 5791, 5835], 211165.2: [147, 206, 276, 312, 365, 438, 509, 655, 667, 794, 1120, 1172, 1234, 1335, 1337, 1762, 1803, 2042, 2239, 2380, 2410, 2513, 2812, 2935, 3193, 3227, 3280, 3303, 3340, 3471, 3473, 3793, 3805, 3824, 3954, 3955, 3972, 4147, 4317, 4396, 4477, 4489, 4557, 4701, 4802, 4862, 4925, 5039, 5204, 5279, 5338, 5506, 5566, 6027, 6083, 6213, 6277, 6355, 6359, 6

## Ordenamiento de los indices de los genes duplicados los diccionarios `genes_secundarios_k_means` y `genes_secundarios_umbrales`

### k-means

In [10]:
genes_secundarios_k_means_sorted = {key: sorted(value) for key, value in genes_secundarios_k_means.items()}
print(genes_secundarios_k_means_sorted)

{2490939.1: [317, 321, 322, 334, 503, 608, 993, 1308, 1494, 1565, 2005, 2008, 2193, 2199, 2660, 2792, 2955, 3024, 3350, 3382, 3596, 3719, 3792, 3793, 3970, 4309, 4507, 4588, 4614, 4940, 5162, 5175, 5508, 5720, 5722, 5959, 6197, 6231], 103690.82: [665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2105, 2315, 2460, 2688, 2936, 2956, 3081, 3286, 3468, 3469, 4855, 4867, 4870, 5161, 5392, 5515, 5803, 5805, 5835], 211165.2: [198, 276, 365, 438, 653, 667, 1120, 1234, 1335, 1337, 1568, 1762, 1803, 2208, 2238, 2239, 2290, 2354, 2380, 2812, 2935, 3193, 3227, 3340, 3471, 3473, 3687, 3793, 3806, 3824, 3954, 3972, 4147, 4203, 4317, 4367, 4396, 4489, 4557, 4701, 4802, 4805, 4862, 5039, 5204, 5506, 6213, 6355, 6359, 6372, 6425, 6443, 6444, 6483, 6608, 6648, 6854, 6868, 6969, 7210], 1472755.9: [88, 430, 868, 915, 921, 1182, 1346, 1623, 1739, 2005, 2257, 3031, 3250, 3334, 3561, 4003, 4343, 4345, 4763, 5327, 5612, 5747, 6272, 6674, 6826, 7090, 7114, 7122, 7160, 7307], 2576904.6: [14, 833, 899, 1

### umbrales

In [7]:
genes_secundarios_umbrales_sorted = {key: sorted(value) for key, value in genes_secundarios_umbrales.items()}
print(genes_secundarios_umbrales_sorted)

{2490939.1: [248, 317, 321, 322, 334, 378, 503, 525, 608, 895, 993, 1106, 1234, 1494, 1565, 1597, 2005, 2144, 2186, 2194, 2199, 2660, 2764, 2882, 2955, 3024, 3350, 3382, 3461, 3655, 3700, 3719, 3792, 3793, 3796, 3970, 4309, 4507, 4588, 4614, 4940, 5063, 5150, 5162, 5173, 5175, 5381, 5518, 5578, 5720, 5722, 5959, 6151, 6197, 6231, 6283, 6284, 6349], 103690.82: [98, 189, 190, 378, 469, 665, 693, 1148, 1286, 1512, 1631, 1655, 1672, 1809, 2064, 2086, 2105, 2315, 2460, 2688, 2766, 2936, 2956, 3002, 3081, 3195, 3286, 3468, 3469, 3472, 3552, 3998, 4086, 4564, 4646, 4855, 4867, 4870, 5161, 5220, 5392, 5435, 5515, 5579, 5611, 5619, 5791, 5835], 211165.2: [147, 206, 276, 312, 365, 438, 509, 655, 667, 794, 1120, 1172, 1234, 1335, 1337, 1762, 1803, 2042, 2239, 2380, 2410, 2513, 2812, 2935, 3193, 3227, 3280, 3303, 3340, 3471, 3473, 3793, 3805, 3824, 3954, 3955, 3972, 4147, 4317, 4396, 4477, 4489, 4557, 4701, 4802, 4862, 4925, 5039, 5204, 5279, 5338, 5506, 5566, 6027, 6083, 6213, 6277, 6355, 6359, 6

## Conteo del número de genes de cada genoma
En el archivo .txt los tres tipos que aparecen (peg, rna y repeat) corresponden a diferentes tipos de características funcionales o estructurales identificadas en los genomas

    peg: Genes codificantes de proteínas.
    rna: Genes que codifican ARN.
    repeat: Secuencias repetitivas en el genoma.(Los repeat no son genes en sí mismos, sino secuencias repetitivas, por lo que no se deben contar como genes.)

¿Cómo contar solo los genes?

Si queremos contar únicamente  los genes funcionales, sólo se deben contar únicamente los registros de tipo peg.

cat 103690.82.txt | grep 'peg' | wc -l

Este comando te dará el número total de genes codificadores de proteínas (PEG) en el archivo.

Incluir los genes de ARN

Si también te interesa contar los genes de ARN (por ejemplo, los tRNA o rRNA), puedes incluir los registros de tipo rna:

cat 103690.82.txt | grep -E 'peg|rna' | wc -l


### nosotros sólo contaremos los genes funcionales

In [9]:
# tabla que contiene el numero de genes
numero_genes = pd.read_csv('numero_de_genes.csv')
print(numero_genes)

     genome_id  number_of_genes
0    103690.82             5854
1   1472755.90             7399
2   1618022.90             7179
3   1647413.14             5966
4   1751286.15             5882
5   1869241.20             6531
6   1914872.23             5088
7   2038116.21             9232
8    211165.20             7334
9   2490939.10             6410
10  2572090.70             7690
11  2576902.60             7330
12  2576903.50             7252
13  2576904.60             6433
14  2653204.70             6637
15   272123.44             6072
16  2764711.14             6648
17    28072.26             6098
18  2914041.10             5991
19  3025190.14             6937
20  3134896.70             8176
21   317936.27             5929
22  3349875.40             7207
23  3349876.50             6605
24   446679.11             6802
25   449208.14             7442
26    46234.36             4261
27    63737.69             7857
28    76335.23             6838


# Función para calcular medir la distancia en número de genes entre genes duplicados

In [12]:
# Función para calcular las diferencias considerando la circularidad
def calcular_diferencias_circulares(posiciones, num_genes):
    # Ordenamos las posiciones de menor a mayor
    posiciones_ordenadas = sorted(posiciones)
    
    diferencias = []
    # Calculamos las diferencias entre un gen y el siguiente considerando la circularidad
    for i in range(1, len(posiciones_ordenadas)):
        diferencia = (posiciones_ordenadas[i] - posiciones_ordenadas[i - 1])
        diferencias.append(diferencia)
    
    # Diferencia circular entre el último gen y el primero
    diferencia_circular = (posiciones_ordenadas[0] - posiciones_ordenadas[-1] + num_genes)
    diferencias.append(diferencia_circular)
    
    return diferencias

### K-means

In [13]:
# Calculamos las diferencias para cada genoma con circularidad
resultados_diferencias_k_means = {}

for genome_id, posiciones in genes_secundarios_k_means_sorted.items():
    num_genes = numero_genes[numero_genes["genome_id"] == genome_id]["number_of_genes"].item()# Usamos el número total de genes
    # Calculamos las diferencias para cada genoma considerando la circularidad
    resultados_diferencias_k_means[genome_id] = calcular_diferencias_circulares(posiciones, num_genes)

print(resultados_diferencias_k_means)

{2490939.1: [4, 1, 12, 169, 105, 385, 315, 186, 71, 440, 3, 185, 6, 461, 132, 163, 69, 326, 32, 214, 123, 73, 1, 177, 339, 198, 81, 26, 326, 222, 13, 333, 212, 2, 237, 238, 34, 496], 103690.82: [28, 455, 138, 226, 119, 24, 17, 137, 255, 41, 210, 145, 228, 248, 20, 125, 205, 182, 1, 1386, 12, 3, 291, 231, 123, 288, 2, 30, 684], 211165.2: [78, 89, 73, 215, 14, 453, 114, 101, 2, 231, 194, 41, 405, 30, 1, 51, 64, 26, 432, 123, 258, 34, 113, 131, 2, 214, 106, 13, 18, 130, 18, 175, 56, 114, 50, 29, 93, 68, 144, 101, 3, 57, 177, 165, 302, 707, 142, 4, 13, 53, 18, 1, 39, 125, 40, 206, 14, 101, 241, 322], 1472755.9: [342, 438, 47, 6, 261, 164, 277, 116, 266, 252, 774, 219, 84, 227, 442, 340, 2, 418, 564, 285, 135, 525, 402, 152, 264, 24, 8, 38, 147, 180], 2576904.6: [819, 66, 123, 565, 2, 53, 204, 220, 224, 94, 3, 41, 21, 1, 182, 11, 71, 1688, 462, 5, 5, 10, 217, 220, 104, 132, 39, 157, 156, 427, 111], 2764711.14: [6, 3, 27, 52, 164, 277, 1, 15, 270, 370, 230, 34, 51, 479, 63, 87, 320, 58, 109,

### umbrales

In [14]:
# Calculamos las diferencias para cada genoma con circularidad
resultados_diferencias_umbrales = {}

for genome_id, posiciones in genes_secundarios_umbrales_sorted.items():
    num_genes = numero_genes[numero_genes["genome_id"] == genome_id]["number_of_genes"].item()# Usamos el número total de genes
    # Calculamos las diferencias para cada genoma considerando la circularidad
    resultados_diferencias_umbrales[genome_id] = calcular_diferencias_circulares(posiciones, num_genes)

print(resultados_diferencias_umbrales)

{2490939.1: [69, 4, 1, 12, 44, 125, 22, 83, 287, 98, 113, 128, 260, 71, 32, 408, 139, 42, 8, 5, 461, 104, 118, 73, 69, 326, 32, 79, 194, 45, 19, 73, 1, 3, 174, 339, 198, 81, 26, 326, 123, 87, 12, 11, 2, 206, 137, 60, 142, 2, 237, 192, 46, 34, 52, 1, 65, 309], 103690.82: [91, 1, 188, 91, 196, 28, 455, 138, 226, 119, 24, 17, 137, 255, 22, 19, 210, 145, 228, 78, 170, 20, 46, 79, 114, 91, 182, 1, 3, 80, 446, 88, 478, 82, 209, 12, 3, 291, 59, 172, 43, 80, 64, 32, 8, 172, 44, 117], 211165.2: [59, 70, 36, 53, 73, 71, 146, 12, 127, 326, 52, 62, 101, 2, 425, 41, 239, 197, 141, 30, 103, 299, 123, 258, 34, 53, 23, 37, 131, 2, 320, 12, 19, 130, 1, 17, 175, 170, 79, 81, 12, 68, 144, 101, 60, 63, 114, 165, 75, 59, 168, 60, 461, 56, 130, 64, 78, 4, 13, 111, 100, 25, 40, 36, 170, 14, 40, 39, 1, 21, 241, 271], 1472755.9: [320, 96, 364, 47, 6, 261, 6, 158, 15, 133, 129, 70, 23, 23, 137, 1, 77, 51, 249, 106, 24, 507, 39, 74, 330, 227, 442, 340, 420, 98, 216, 14, 236, 65, 220, 135, 329, 196, 173, 79, 150,

### Ejemplo de como funciona el código

In [22]:
# Datos de prueba
genome_id = 103690.82
genes_copia_prueba =  {
    103690.82: [446, 1364, 1440, 2280, 2281, 2547, 2562, 2772, 
                3614, 3879, 4087, 4164, 4265, 4537, 4984, 5106, 5242, 5250]
}
num_genes = numero_genes.loc[numero_genes['genome_id'] == 103690.82, 'number_of_genes'].item()
print("genome_id:",genome_id)
print("genes copia:",genes_copia_prueba)
print("número total de genes:",num_genes)

resultado_test = {}
# Ejecutamos la función
resultado_test[genome_id] = calcular_diferencias_circulares(genes_copia_prueba[genome_id], num_genes)

# Mostramos el resultado
print("diferencias en número de genes",resultado_test)


genome_id: 103690.82
genes copia: {103690.82: [446, 1364, 1440, 2280, 2281, 2547, 2562, 2772, 3614, 3879, 4087, 4164, 4265, 4537, 4984, 5106, 5242, 5250]}
número total de genes: 5854
diferencias en número de genes {103690.82: [918, 76, 840, 1, 266, 15, 210, 842, 265, 208, 77, 101, 272, 447, 122, 136, 8, 1050]}


## Estadísticas descriptivas para cada uno de los 29 genomas

### k-means

In [21]:
# Crear un nuevo diccionario para almacenar las estadísticas
estadisticas_k_means = {}

# Calcular estadísticas para cada genoma
for id_genoma, diferencias in resultados_diferencias_k_means.items():
    diferencias = np.array(diferencias)  # Convertir a un array de NumPy para evitar errores
    estadisticas_k_means[id_genoma] = {
        'media (KB)': np.mean(diferencias) if diferencias.size > 0 else np.nan,
        'mediana (KB)': np.median(diferencias) if diferencias.size > 0 else np.nan,
        'desviación estándar (KB)': np.std(diferencias, ddof=1) if diferencias.size > 1 else np.nan,
        'máximo (KB)': np.max(diferencias) if diferencias.size > 0 else np.nan,
        'mínimo (KB)': np.min(diferencias) if diferencias.size > 0 else np.nan
    }

# Convertir a DataFrame
df_estadisticas_k_means = pd.DataFrame.from_dict(estadisticas_k_means, orient='index')

# Mostrar el DataFrame
df_estadisticas_k_means

,media (KB),mediana (KB),desviación estándar (KB),máximo (KB),mínimo (KB)
2490939.10,168.684211,166.0,144.494479,496,1
103690.82,201.862069,138.0,273.374511,1386,1
211165.20,122.233333,91.0,132.820685,707,1
1472755.90,246.633333,239.5,185.756326,774,2
2576904.60,207.516129,111.0,331.592206,1688,1
2764711.14,154.604651,96.0,151.419811,613,1
3134896.70,157.230769,96.0,173.227111,834,1
3349875.40,218.393939,122.0,237.516702,927,2
63737.69,196.425000,148.5,179.480574,745,1
272123.44,216.857143,81.0,348.121064,1391,1


### umbrales

In [22]:
# Crear un nuevo diccionario para almacenar las estadísticas
estadisticas_umbrales = {}

# Calcular estadísticas para cada genoma
for id_genoma, diferencias in resultados_diferencias_umbrales.items():
    diferencias = np.array(diferencias)  # Asegurar que sea un array de NumPy
    estadisticas_umbrales[id_genoma] = {
        'media (KB)': np.mean(diferencias) if diferencias.size > 0 else np.nan,
        'mediana (KB)': np.median(diferencias) if diferencias.size > 0 else np.nan,
        'desviación estándar (KB)': np.std(diferencias, ddof=1) if diferencias.size > 1 else np.nan,
        'máximo (KB)': np.max(diferencias) if diferencias.size > 0 else np.nan,
        'mínimo (KB)': np.min(diferencias) if diferencias.size > 0 else np.nan
    }

# Convertir a DataFrame
df_estadisticas_umbrales = pd.DataFrame.from_dict(estadisticas_umbrales, orient='index')

# Mostrar el Data
df_estadisticas_umbrales

,media (KB),mediana (KB),desviación estándar (KB),máximo (KB),mínimo (KB)
2490939.10,110.517241,73.0,111.975286,461,1
103690.82,121.958333,89.5,116.484308,478,1
211165.20,101.861111,69.0,98.578960,461,1
1472755.90,147.980000,131.0,130.114312,507,1
2576904.60,142.955556,71.0,148.203017,484,2
2764711.14,110.800000,75.5,117.520254,471,1
3134896.70,125.784615,77.0,138.762555,588,2
3349875.40,141.313725,106.0,154.955799,764,1
63737.69,140.303571,92.0,149.083737,735,1
76335.23,129.018868,93.0,119.838592,460,1


## Estadisticas globales

### k-means

In [19]:
# Concatenar todas las diferencias en una sola lista
todas_diferencias_k_means = np.concatenate(list(resultados_diferencias_k_means.values()))

# Calcular estadísticas globales
estadisticas_globales_k_means = {
    'media (KB)': np.mean(todas_diferencias_k_means),
    'mediana (KB)': np.median(todas_diferencias_k_means),
    'desviación estándar (KB)': np.std(todas_diferencias_k_means),
    'máximo (KB)': np.max(todas_diferencias_k_means),
    'mínimo (KB)': np.min(todas_diferencias_k_means)
}

# Mostrar estadísticas globales
print('Estadísticas globales:')
for key, value in estadisticas_globales_k_means.items():
    print(f'  {key}: {value:.2f}')

Estadísticas globales:
  media (KB): 209.61
  mediana (KB): 134.50
  desviación estándar (KB): 250.29
  máximo (KB): 2117.00
  mínimo (KB): 1.00


## umbrales

In [20]:
# Concatenar todas las diferencias en una sola lista
todas_diferencias_umbrales = np.concatenate(list(resultados_diferencias_umbrales.values()))

# Calcular estadísticas globales
estadisticas_globales_umbrales = {
    'media (KB)': np.mean(todas_diferencias_umbrales),
    'mediana (KB)': np.median(todas_diferencias_umbrales),
    'desviación estándar (KB)': np.std(todas_diferencias_umbrales),
    'máximo (KB)': np.max(todas_diferencias_umbrales),
    'mínimo (KB)': np.min(todas_diferencias_umbrales)
}

# Mostrar estadísticas globales
print('Estadísticas globales:')
for key, value in estadisticas_globales_umbrales.items():
    print(f'  {key}: {value:.2f}')

Estadísticas globales:
  media (KB): 145.30
  mediana (KB): 93.00
  desviación estándar (KB): 159.60
  máximo (KB): 1309.00
  mínimo (KB): 1.00


## Exportar los datos 

### k-means

In [25]:
## Guardar el df * * en un archivo .csv para poder importarlo para el siguiente notebook
# Guardar como pkl
# Guardar el diccionario en un archivo
with open('resultados_en_numero_genes_k_means.pkl', 'wb') as file:
    pickle.dump(resultados_diferencias_k_means, file)

### umbrales

In [26]:
## Guardar el df * * en un archivo .csv para poder importarlo para el siguiente notebook
# Guardar como pkl
# Guardar el diccionario en un archivo
with open('resultados_en_numero_genes_umbrales.pkl', 'wb') as file:
    pickle.dump(resultados_diferencias_umbrales, file)

In [27]:
%autosave 30

Autosaving every 30 seconds
